# Phase 5 Capstone

# Predicting Obesity/ Obesity Prediction

- Student name: Lucas Wilkerson
- Student pace: Flex/part time
- Scheduled project review date/time:
- Instructor name: Morgan Jones

## Business Understanding 

## Data Understanding 

## Data Preparation

## Modeling

## Evaluation

## Deployment

In [1]:
# Importing standard Packages 
import pandas as pd
import numpy as np
import math 
import zipfile

# Importing packages for visualization 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Importing packages for statistics
!pip install statsmodels
import statsmodels.api as sm
from scipy import stats
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import sklearn.metrics as metrics
from scipy.stats import kstest

import os

In [3]:
# Function to load dataset 

def load_csv_from_zip(zip_file_path, csv_file_name):
    
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        # Check if the CSV file exists in the zip file
        if csv_file_name in zip_ref.namelist():
            # Read the CSV file directly into a DataFrame
            with zip_ref.open(csv_file_name) as csv_file:
                df = pd.read_csv(csv_file)
                return df
        else:
            print(f"CSV file '{csv_file_name}' not found in the zip file.")
            return None

zip_file_path = "data/brfss2022.csv.zip"
csv_file_name = "brfss2022.csv"
df = load_csv_from_zip(zip_file_path, csv_file_name)
if df is not None:
    print("CSV file loaded successfully!")
    print(df.head())  # Display the first few rows of the DataFrame

CSV file loaded successfully!
   _STATE  FMONTH    IDATE  IMONTH  IDAY  IYEAR  DISPCODE       SEQNO  \
0     1.0     1.0  2032022       2     3   2022    1100.0  2022000001   
1     1.0     1.0  2042022       2     4   2022    1100.0  2022000002   
2     1.0     1.0  2022022       2     2   2022    1100.0  2022000003   
3     1.0     1.0  2032022       2     3   2022    1100.0  2022000004   
4     1.0     1.0  2022022       2     2   2022    1100.0  2022000005   

           _PSU  CTELENM1  ...  _SMOKGRP  _LCSREC  DRNKANY6      DROCDY4_  \
0  2.022000e+09       1.0  ...       4.0      NaN       2.0  5.397605e-79   
1  2.022000e+09       1.0  ...       4.0      NaN       2.0  5.397605e-79   
2  2.022000e+09       1.0  ...       4.0      NaN       2.0  5.397605e-79   
3  2.022000e+09       1.0  ...       3.0      2.0       2.0  5.397605e-79   
4  2.022000e+09       1.0  ...       4.0      NaN       1.0  1.000000e+01   

   _RFBING6      _DRNKWK2  _RFDRHV8  _FLSHOT7  _PNEUMO3  _AIDTST4  


In [4]:
# Previewing general information on dataset
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 445132 entries, 0 to 445131
Columns: 326 entries, _STATE to _AIDTST4
dtypes: float64(321), int64(5)
memory usage: 1.1 GB
None


Looking at the initial dataset information, this dataset contains 445132 entries with a total of 326 columns with datatypes being integers and floats. While there are some numerical columns, a good amount of the columns are categoricla/numerical. The many columns represent  various health-related risk behaviors, chronic health conditions, demographics, location, and even the use of preventive services along with other variables. 

In [5]:
#Previewing first 10 rows 
df.head(10)


,_STATE,FMONTH,IDATE,IMONTH,IDAY,IYEAR,DISPCODE,SEQNO,_PSU,CTELENM1,...,_SMOKGRP,_LCSREC,DRNKANY6,DROCDY4_,_RFBING6,_DRNKWK2,_RFDRHV8,_FLSHOT7,_PNEUMO3,_AIDTST4
0,1.0,1.0,2032022,2,3,2022,1100.0,2022000001,2.022000e+09,1.0,...,4.0,NaN,2.0,5.397605e-79,1.0,5.397605e-79,1.0,1.0,2.0,2.0
1,1.0,1.0,2042022,2,4,2022,1100.0,2022000002,2.022000e+09,1.0,...,4.0,NaN,2.0,5.397605e-79,1.0,5.397605e-79,1.0,2.0,2.0,2.0
2,1.0,1.0,2022022,2,2,2022,1100.0,2022000003,2.022000e+09,1.0,...,4.0,NaN,2.0,5.397605e-79,1.0,5.397605e-79,1.0,NaN,NaN,2.0
3,1.0,1.0,2032022,2,3,2022,1100.0,2022000004,2.022000e+09,1.0,...,3.0,2.0,2.0,5.397605e-79,1.0,5.397605e-79,1.0,9.0,9.0,2.0
4,1.0,1.0,2022022,2,2,2022,1100.0,2022000005,2.022000e+09,1.0,...,4.0,NaN,1.0,1.000000e+01,1.0,1.400000e+02,1.0,NaN,NaN,2.0
5,1.0,1.0,2112022,2,11,2022,1100.0,2022000006,2.022000e+09,1.0,...,4.0,NaN,2.0,5.397605e-79,1.0,5.397605e-79,1.0,2.0,1.0,2.0
6,1.0,1.0,2032022,2,3,2022,1100.0,2022000007,2.022000e+09,1.0,...,3.0,NaN,1.0,1.700000e+01,1.0,2.330000e+02,1.0,2.0,2.0,2.0
7,1.0,1.0,2092022,2,9,2022,1100.0,2022000008,2.022000e+09,1.0,...,4.0,NaN,2.0,5.397605e-79,1.0,5.397605e-79,1.0,1.0,1.0,2.0
8,1.0,1.0,2022022,2,2,2022,1100.0,2022000009,2.022000e+09,1.0,...,3.0,NaN,2.0,5.397605e-79,1.0,5.397605e-79,1.0,2.0,2.0,1.0
9,1.0,1.0,2072022,2,7,2022,1100.0,2022000010,2.022000e+09,1.0,...,4.0,NaN,1.0,3.000000e+00,1.0,2.300000e+01,1.0,1.0,1.0,9.0


In [6]:
#Previewing first 10 rows 
df.head(10).T

,0,1,2,3,4,5,6,7,8,9
_STATE,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.0,1.000000e+00,1.0,1.000000e+00,1.000000e+00,1.0
FMONTH,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.0,1.000000e+00,1.0,1.000000e+00,1.000000e+00,1.0
IDATE,2.032022e+06,2.042022e+06,2.022022e+06,2.032022e+06,2022022.0,2.112022e+06,2032022.0,2.092022e+06,2.022022e+06,2072022.0
IMONTH,2.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.0,2.000000e+00,2.0,2.000000e+00,2.000000e+00,2.0
IDAY,3.000000e+00,4.000000e+00,2.000000e+00,3.000000e+00,2.0,1.100000e+01,3.0,9.000000e+00,2.000000e+00,7.0
...,...,...,...,...,...,...,...,...,...,...
_DRNKWK2,5.397605e-79,5.397605e-79,5.397605e-79,5.397605e-79,140.0,5.397605e-79,233.0,5.397605e-79,5.397605e-79,23.0
_RFDRHV8,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.0,1.000000e+00,1.0,1.000000e+00,1.000000e+00,1.0
_FLSHOT7,1.000000e+00,2.000000e+00,NaN,9.000000e+00,NaN,2.000000e+00,2.0,1.000000e+00,2.000000e+00,1.0
_PNEUMO3,2.000000e+00,2.000000e+00,NaN,9.000000e+00,NaN,1.000000e+00,2.0,1.000000e+00,2.000000e+00,1.0


In [7]:
# Previewing columns 
print(df.columns)

Index(['_STATE', 'FMONTH', 'IDATE', 'IMONTH', 'IDAY', 'IYEAR', 'DISPCODE',
       'SEQNO', '_PSU', 'CTELENM1',
       ...
       '_SMOKGRP', '_LCSREC', 'DRNKANY6', 'DROCDY4_', '_RFBING6', '_DRNKWK2',
       '_RFDRHV8', '_FLSHOT7', '_PNEUMO3', '_AIDTST4'],
      dtype='object', length=326)


In [13]:
#Looping to get all column names and data types
for col in df.columns:
    print(f"Column: {col}, Data Type: {df[col].dtype}")

Column: _STATE, Data Type: float64
Column: FMONTH, Data Type: float64
Column: IDATE, Data Type: int64
Column: IMONTH, Data Type: int64
Column: IDAY, Data Type: int64
Column: IYEAR, Data Type: int64
Column: DISPCODE, Data Type: float64
Column: SEQNO, Data Type: int64
Column: _PSU, Data Type: float64
Column: CTELENM1, Data Type: float64
Column: PVTRESD1, Data Type: float64
Column: COLGHOUS, Data Type: float64
Column: STATERE1, Data Type: float64
Column: CELPHON1, Data Type: float64
Column: LADULT1, Data Type: float64
Column: COLGSEX1, Data Type: float64
Column: NUMADULT, Data Type: float64
Column: LANDSEX1, Data Type: float64
Column: NUMMEN, Data Type: float64
Column: NUMWOMEN, Data Type: float64
Column: RESPSLCT, Data Type: float64
Column: SAFETIME, Data Type: float64
Column: CTELNUM1, Data Type: float64
Column: CELLFON5, Data Type: float64
Column: CADULT1, Data Type: float64
Column: CELLSEX1, Data Type: float64
Column: PVTRESD3, Data Type: float64
Column: CCLGHOUS, Data Type: float64
C

In [10]:
#Inspecting unique values and value counts for all variables
for col in df:
    print(f"{col} - unique values: ")
    print(df[col].unique(), "\n")
    print("------------------------------------")

_STATE - unique values: 
[ 1.  2.  4.  5.  6.  8.  9. 10. 11. 12. 13. 15. 16. 17. 18. 19. 20. 21.
 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35. 36. 37. 38. 39.
 40. 41. 42. 44. 45. 46. 47. 48. 49. 50. 51. 53. 54. 55. 56. 66. 72. 78.] 

------------------------------------
FMONTH - unique values: 
[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12.] 

------------------------------------
IDATE - unique values: 
[ 2032022  2042022  2022022  2112022  2092022  2072022  2102022  2182022
  2162022  2152022  2242022  2172022  3142022  3032022  3082022  3112022
  3102022  3092022  2142022  2212022  2082022  2232022  2222022  2252022
  3152022  3022022  3072022  2052022  3162022  3042022  3172022  2132022
  3182022  4192022  4202022  4182022  4222022  4272022  4282022  4262022
  5022022  5122022  5042022  5102022  5172022  5262022  5232022  6012022
  6302022  6242022  6232022  6212022  6172022  6162022  6152022  5032022
  5252022  6282022  5312022  4252022  5052022  6272022  6222022  42

In [12]:
# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# Print the full samples
print(df.sample(1))

        _STATE  FMONTH    IDATE  IMONTH  IDAY  IYEAR  DISPCODE       SEQNO  \
140346    22.0     1.0  4082022       4     8   2022    1200.0  2022001760   

                _PSU  CTELENM1  PVTRESD1  COLGHOUS  STATERE1  CELPHON1  \
140346  2.022002e+09       NaN       NaN       NaN       NaN       NaN   

        LADULT1  COLGSEX1  NUMADULT  LANDSEX1  NUMMEN  NUMWOMEN  RESPSLCT  \
140346      NaN       NaN       NaN       NaN     NaN       NaN       NaN   

        SAFETIME  CTELNUM1  CELLFON5  CADULT1  CELLSEX1  PVTRESD3  CCLGHOUS  \
140346       1.0       1.0       1.0      1.0       2.0       1.0       NaN   

        CSTATE1  LANDLINE  HHADULT  SEXVAR  GENHLTH  PHYSHLTH  MENTHLTH  \
140346      1.0       2.0      5.0     2.0      2.0       2.0       2.0   

        POORHLTH  PRIMINSR  PERSDOC3  MEDCOST1  CHECKUP1  EXERANY2  SLEPTIM1  \
140346       1.0       1.0       2.0       2.0       1.0       2.0       7.0   

        LASTDEN4  RMVTETH4  CVDINFR4  CVDCRHD4  CVDSTRK3  ASTHMA3  A

In [14]:
# Checking for null values
df.isna().sum()

_STATE           0
FMONTH           0
IDATE            0
IMONTH           0
IDAY             0
             ...  
_DRNKWK2         0
_RFDRHV8         0
_FLSHOT7    276713
_PNEUMO3    276713
_AIDTST4     46235
Length: 326, dtype: int64

In [16]:
# Checking null values for columns

na_values = pd.DataFrame(df.isna().sum(), columns=['Total Values null'])
na_values['%_weight'] = na_values['Total Values null'].apply(lambda x: x/df.shape[0]*100)
na_values

,Total Values null,%_weight
_STATE,0,0.000000
FMONTH,0,0.000000
IDATE,0,0.000000
IMONTH,0,0.000000
IDAY,0,0.000000
...,...,...
_DRNKWK2,0,0.000000
_RFDRHV8,0,0.000000
_FLSHOT7,276713,62.164257
_PNEUMO3,276713,62.164257


In [18]:
na_over_half = na_values[na_values["%_weight"] > 50]
na_over_half

,Total Values null,%_weight
CTELENM1,349081,78.421906
PVTRESD1,349082,78.422131
COLGHOUS,445113,99.995732
STATERE1,349082,78.422131
CELPHON1,349080,78.421682
...,...,...
_PACKYRS,302267,67.905026
_YRSQUIT,338802,76.112704
_LCSREC,393679,88.440957
_FLSHOT7,276713,62.164257


In [19]:
# Columns with greater than 50% na values 
na_over_half.index.values

array(['CTELENM1', 'PVTRESD1', 'COLGHOUS', 'STATERE1', 'CELPHON1',
       'LADULT1', 'COLGSEX1', 'NUMADULT', 'LANDSEX1', 'NUMMEN',
       'NUMWOMEN', 'RESPSLCT', 'CCLGHOUS', 'ASTHNOW', 'DIABAGE4',
       'NUMHHOL4', 'NUMPHON4', 'PREGNANT', 'HADMAM', 'HOWLONG',
       'CERVSCRN', 'CRVCLCNC', 'CRVCLPAP', 'CRVCLHPV', 'HADHYST2',
       'COLNSIGM', 'COLNTES1', 'SIGMTES1', 'LASTSIG4', 'VIRCOLO1',
       'VCLNTES2', 'SMALSTOL', 'STOLTEST', 'STOOLDN2', 'BLDSTFIT',
       'SDNATES1', 'SMOKDAY2', 'LCSFIRST', 'LCSLAST', 'LCSNUMCG',
       'LCSSCNCR', 'LCSCTWHN', 'AVEDRNK3', 'DRNK3GE5', 'MAXDRNKS',
       'FLSHTMY3', 'HIVTSTD3', 'COVIDSMP', 'COVIDPRM', 'PDIABTS1',
       'PREDIAB2', 'DIABTYPE', 'INSULIN1', 'CHKHEMO3', 'EYEEXAM1',
       'DIABEYE1', 'DIABEDU1', 'FEETSORE', 'TOLDCFS', 'HAVECFS',
       'WORKCFS', 'IMFVPLA3', 'HPVADVC4', 'HPVADSHT', 'SHINGLE2',
       'COVIDVA1', 'COVIDNU1', 'COVIDFS1', 'COVIDSE1', 'COPDCOGH',
       'COPDFLEM', 'COPDBRTH', 'COPDBTST', 'COPDSMOK', 'CNCRDIFF',
      

In [20]:
na_over_half.T

,CTELENM1,PVTRESD1,COLGHOUS,STATERE1,CELPHON1,LADULT1,COLGSEX1,NUMADULT,LANDSEX1,NUMMEN,NUMWOMEN,RESPSLCT,CCLGHOUS,ASTHNOW,DIABAGE4,NUMHHOL4,NUMPHON4,PREGNANT,HADMAM,HOWLONG,CERVSCRN,CRVCLCNC,CRVCLPAP,CRVCLHPV,HADHYST2,COLNSIGM,COLNTES1,SIGMTES1,LASTSIG4,VIRCOLO1,VCLNTES2,SMALSTOL,STOLTEST,STOOLDN2,BLDSTFIT,SDNATES1,SMOKDAY2,LCSFIRST,LCSLAST,LCSNUMCG,LCSSCNCR,LCSCTWHN,AVEDRNK3,DRNK3GE5,MAXDRNKS,FLSHTMY3,HIVTSTD3,COVIDSMP,COVIDPRM,PDIABTS1,PREDIAB2,DIABTYPE,INSULIN1,CHKHEMO3,EYEEXAM1,DIABEYE1,DIABEDU1,FEETSORE,TOLDCFS,HAVECFS,WORKCFS,IMFVPLA3,HPVADVC4,HPVADSHT,SHINGLE2,COVIDVA1,COVIDNU1,COVIDFS1,COVIDSE1,COPDCOGH,COPDFLEM,COPDBRTH,COPDBTST,COPDSMOK,CNCRDIFF,CNCRAGE,CNCRTYP2,CSRVTRT3,CSRVDOC1,CSRVSUM,CSRVRTRN,CSRVINST,CSRVINSR,CSRVDEIN,CSRVCLIN,CSRVPAIN,CSRVCTL2,PSATEST1,PSATIME1,PCPSARS2,PSASUGST,PCSTALK1,CIMEMLOS,CDHOUSE,CDASSIST,CDHELP,CDSOCIAL,CDDISCUS,CAREGIV1,CRGVREL4,CRGVLNG1,CRGVHRS1,CRGVPRB3,CRGVALZD,CRGVPER1,CRGVHOU1,CRGVEXPT,ACEDEPRS,ACEDRINK,ACEDRUGS,ACEPRISN,ACEDIVRC,ACEPUNCH,ACEHURT1,ACESWEAR,ACETOUCH,ACETTHEM,ACEHVSEX,ACEADSAF,ACEADNED,MARIJAN1,MARJSMOK,MARJEAT,MARJVAPE,MARJDAB,MARJOTHR,USEMRJN4,LASTSMK2,STOPSMK2,MENTCIGS,MENTECIG,HEATTBCO,ASBIALCH,ASBIDRNK,ASBIBING,ASBIADVC,ASBIRDUC,FIREARM5,GUNLOAD,LOADULK2,RCSGEND1,RCSXBRTH,RCSRLTN2,CASTHDX2,CASTHNO2,BIRTHSEX,SOMALE,SOFEMALE,HADSEX,PFPPRVN4,TYPCNTR9,BRTHCNT4,WHEREGET,NOBCUSE8,BCPREFER,RRCLASS3,RRCOGNT2,RRTREAT,RRATWRK2,RRHCARE4,RRPHYSM2,MSCODE,_CRACE2,_CPRACE2,CAGEG,_CLLCPWT,_DUALCOR,_ALTETH3,_RFMAM22,_MAM5023,_CLNSCP1,_SGMSCP1,_SGMS101,_RFBLDS5,_STOLDN1,_VIRCOL1,_YRSSMOK,_PACKDAY,_PACKYRS,_YRSQUIT,_LCSREC,_FLSHOT7,_PNEUMO3
Total Values null,349081.000000,349082.000000,445113.000000,349082.000000,349080.000000,349082.000000,445111.000000,349100.000000,401436.000000,394702.000000,394699.000000,392879.000000,443807.000000,378438.000000,383973.00000,349083.000000,438189.000000,366114.000000,223319.000000,274020.000000,224318.000000,309555.000000,309745.000000,309960.000000,227453.000000,232309.000000,237848.000000,413769.000000,441658.000000,383265.000000,436153.000000,383456.000000,395758.000000,383730.000000,433738.00000,433770.000000,281079.00000,283417.000000,324513.000000,288992.000000,279557.000000,413627.000000,234769.000000,235283.000000,235721.000000,236258.000000,316269.00000,320933.000000,418417.000000,304884.000000,304910.000000,432532.000000,432532.000000,432532.000000,432532.000000,432532.000000,432532.000000,432532.000000,445132.0,445132.0,445132.0,436089.000000,437786.000000,443460.000000,430395.000000,292089.000000,320385.000000,321193.000000,328043.000000,437863.000000,437870.000000,437883.000000,437900.000000,437917.000000,421733.000000,422567.000000,422588.000000,427830.000000,435325.000000,435349.000000,435361.000000,437882.00000,435384.000000,435392.000000,435403.000000,428035.000000,443591.000000,437952.000000,441186.000000,441196.000000,440738.000000,438364.000000,380457.000000,437679.000000,437701.000000,443199.000000,437717.000000,437732.000000,346622.000000,425498.000000,425549.000000,425599.000000,425660.000000,427832.000000,425732.000000,425772.000000,366598.000000,396846.000000,396901.000000,396912.000000,396938.000000,396960.000000,397003.000000,397055.000000,397103.000000,397160.000000,397196.00000,397256.000000,397311.000000,397375.000000,350213.000000,433622.00000,433628.000000,433629.000000,433637.00000,433645.000000,440126.00000,397723.000000,426243.00000,440138.000000,442968.000000,430103.000000,384190.00000,384292.000000,384404.000000,385687.000000,398512.000000,405332.000000,431307.00000,441077.000000,386144.000000,441669.000000,386658.000000,394471.000000,439804.000000,365705.000000,321100.000000,305502.000000,425256.000000,430451.000000,433762.000000,433791.000000,435358.000000,441970.000000,430605.000000,283394.00000,283788.000000,284167.000000,362095.000000,284624.000000,284942.000000,351246.000000,384286.000000,385142.000000,394956.000000,385986.000000,287103.000000,276713.000000,263892.000000,342456.000000,223180.000000

In [23]:
print(df.sample(1))

        _STATE  FMONTH     IDATE  IMONTH  IDAY  IYEAR  DISPCODE       SEQNO  \
262340    36.0    12.0  12212022      12    21   2022    1100.0  2022004620   

                _PSU  CTELENM1  PVTRESD1  COLGHOUS  STATERE1  CELPHON1  \
262340  2.022005e+09       1.0       1.0       NaN       1.0       2.0   

        LADULT1  COLGSEX1  NUMADULT  LANDSEX1  NUMMEN  NUMWOMEN  RESPSLCT  \
262340      1.0       NaN       1.0       2.0     NaN       NaN       NaN   

        SAFETIME  CTELNUM1  CELLFON5  CADULT1  CELLSEX1  PVTRESD3  CCLGHOUS  \
262340       NaN       NaN       NaN      NaN       NaN       NaN       NaN   

        CSTATE1  LANDLINE  HHADULT  SEXVAR  GENHLTH  PHYSHLTH  MENTHLTH  \
262340      NaN       NaN      NaN     2.0      4.0      88.0      88.0   

        POORHLTH  PRIMINSR  PERSDOC3  MEDCOST1  CHECKUP1  EXERANY2  SLEPTIM1  \
262340       NaN       3.0       3.0       2.0       1.0       1.0       6.0   

        LASTDEN4  RMVTETH4  CVDINFR4  CVDCRHD4  CVDSTRK3  ASTHMA3 